## Saúde Bucal - material suplementar


### Procedimentos metodológicos
---
A construção do modelo de planejamento da força de trabalho (PDFT) para saúde bucal baseado em necessidades foi elaborado a partir das orientações de Asamani et al. (2021) que sugerem algumas etapas, que serão descritas com maior detalhamento a seguir:

1. Definição de escopo da metodologia em termos de cobertura jurisdicional, objetivos e horizonte temporal;
2. Análise da necessidade de saúde bucal com base em aspectos da demografia, epidemiologia e serviços destinados da população a local;
3. Tradução do número de serviços em número de profissionais necessários;
4. Exploração das implicações em termos de recursos, como a oferta atual de profissionais e custos;
5. Análise de sensibilidade para simulação de parâmetros de incerteza;
6. Validação de modelo por meio de consulta a stakeholders.

### Preparando ambiente do R
---

Carregando os pacotes necessários para o Python.

In [4]:
!pip install seaborn

     -------------------------------------- 294.9/294.9 kB 2.3 MB/s eta 0:00:00


In [6]:
#instalando os pacotes que ainda não foram instalados

#!pip install pyodbc
#!pip install geopandas
#!pip install rtree
#!pip install pygeos
#!pip install mapclassify
#!pip install geobr



In [12]:
#import numpy as np
print(np.__version__)

2.2.2


In [14]:
!pip uninstall -y numpy

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


In [13]:
!pip list

Package                       Version
----------------------------- -----------
adjustText                    1.3.0
affine                        2.4.0
altair                        5.5.0
anaconda-client               1.11.2
anaconda-navigator            2.4.0
anaconda-project              0.11.1
ansi2html                     1.8.0
anyio                         4.6.2
archspec                      0.2.3
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.0.5
async-lru                     2.0.4
attrs                         24.3.0
babel                         2.16.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
beautifulsoup4                4.12.3
bleach                        6.2.0
blinker                       1.9.0
boltons                       24.1.0
Brotli                        1.0.9
cachetools                    5.5.1
Cartopy                       0.24.1
category-

In [10]:
# Modelagem
import pandas as pd
import numpy as np

#execução dremio
import pyodbc

# Data View
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib.colors import Normalize
from matplotlib import cm
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors

#Mapas
import geopandas as gpd
import rtree
#import pygeos
import mapclassify
import geobr
from matplotlib.ticker import FuncFormatter

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

### 1. Definição de escopo
---
A metodologia tem como objetivo estimar a força de trabalho atual e necessária de cirurgiões dentistas e técnicos e auxilares de saúde bucal para atuação em municípios brasileiros, a nível de atenção primária, em procedimentos de natureza individual, bem como três procedimentos de natureza especializada: endodontia, prótese e periodontia.


### 2. Análise das necessidades de saúde bucal
---
A análise da necessidade de saúde bucal (NSB) foi construída a partir de três elementos: distribuição da população por faixa etária (P), prevalência de condições de saúde bucal (H) e procedimentos per capita (S).

![image.png](imagens/equacao1_sb.png)

Onde:

* NSB = Necessidade de saúde bucal é o número de serviços do tipo t destinados ao atendimento de população de faixa etária i e localidade l;
* P = População por faixa-etária i de uma localidade l;
* H = Prevalência de condições de saúde bucal que levam à necessidade de procedimentos do tipo t por faixa-etária i e dentro de uma localidade l;
* S = Procedimentos per capita do tipo t, para cada faixa-etária i e dentro de uma localidade l.

### 2.1. Levantando a faixa etária (P)

A distribuição por faixa etária foi construída com base em dados do último censo (2022) e estão disponíveis na página do IBGE.

In [2]:
teste = pd.read_excel("populacao_municipio_censo_completa.xlsx")
teste

,cod_municipiodv,municipio,ibge_sb,0 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,...,75 a 79 anos,80 a 84 anos,85 a 89 anos,90 a 94 anos,95 a 99 anos,100 anos ou mais,de_0_a_14_anos,de_15_a_29_anos,de_30_a_59_anos,acima_de_60_anos
0,5300108,Brasília,5300108,166848,184611,182774,208546,227966,224162,223174,...,42396,27501,14401,6121,1720,300,534233,660674,1257384,356949
1,5208707,Goiânia,5208707,83676,89501,85964,94233,113359,120411,118931,...,25381,16036,8393,3555,922,171,259141,328003,633709,211865
2,5200050,Abadia de Goiás,5200000,1590,1619,1601,1434,1449,1521,1645,...,195,114,60,23,10,2,4810,4404,8001,1878
3,5200100,Abadiânia,5200000,1171,1228,1198,1184,1292,1199,1127,...,323,217,90,43,7,4,3597,3675,7146,2760
4,5200134,Acreúna,5200000,1384,1497,1448,1627,1569,1552,1538,...,376,204,115,37,7,6,4329,4748,9326,3115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5764,1101609,Theobroma,1300000,521,533,604,589,452,465,491,...,159,91,44,14,3,1,1658,1506,3533,1398
5765,1101708,Urupá,1300000,741,713,706,811,738,747,773,...,190,134,68,26,3,1,2160,2296,4468,1771
5766,1101757,Vale do Anari,1300000,516,597,598,570,474,512,521,...,135,76,28,15,2,-,1711,1556,3323,1181
5767,1101807,Vale do Paraíso,1300000,381,413,410,472,416,414,382,...,132,73,55,15,8,2,1204,1302,2825,1123
